In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import FirefoxOptions
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

In [ ]:
# Ensure that the driver path is correct before running this script.
# Microsoft Windows
# binary = FirefoxBinary('C:\Program Files\Mozilla Firefox\private_browsing.exe')
driver_path = r"chromedriver.exe"
# opts = FirefoxOptions()
# opts.add_argument('--headless')
# Linux
#driver_path = "./drivers/linux/geckodriver"
driver = webdriver.Chrome(executable_path=driver_path)

In [14]:
## Enter a job position
position = "data scientist"
## Enter a location (City, State or Zip or remote)
locations = "usa"

def get_url(position, location):
    url_template = "https://www.indeed.com/jobs?q={}&l={}"
    url = url_template.format(position, location)
    return url

url = get_url(position, locations)
dataframe = pd.DataFrame(columns=["Title", "Company", "Location", "Rating", "Date", "Salary", "Description", "Links"])

In [15]:
## Number of postings to scrape
postings = 2000

jn=0
for i in range(0, postings, 10):
    driver.get(url + "&start=" + str(i))
    driver.implicitly_wait(3)

    jobs = driver.find_elements(By.CLASS_NAME, 'job_seen_beacon')

    for job in jobs:
        result_html = job.get_attribute('innerHTML')
        soup = BeautifulSoup(result_html, 'html.parser')

        jn += 1

        liens = job.find_elements(By.TAG_NAME, "a")
        links = liens[0].get_attribute("href")

        title = soup.select('.jobTitle')[0].get_text().strip()
        company = soup.select('.companyName')[0].get_text().strip()
        location = soup.select('.companyLocation')[0].get_text().strip()
        try:
            salary = soup.select('.salary-snippet-container')[0].get_text().strip()
        except:
            salary = 'NaN'
        try:
            rating = soup.select('.ratingNumber')[0].get_text().strip()
        except:
            rating = 'NaN'
        try:
            date = soup.select('.date')[0].get_text().strip()
        except:
            date = 'NaN'
        try:
            description = soup.select('.job-snippet')[0].get_text().strip()
        except:
            description = ''

        dataframe = pd.concat([dataframe, pd.DataFrame([{'Title': title,
                                          "Company": company,
                                          'Location': location,
                                          'Rating': rating,
                                          'Date': date,
                                          "Salary": salary,
                                          "Description": description,
                                          "Links": links}])], ignore_index=True)
        print("Job number {0:4d} added - {1:s}".format(jn,title))

Job number    1 added - Jr. Data Scientist
Job number    2 added - Jr. Data Scientist
Job number    3 added - Data Scientist
Job number    4 added - Data Scientist | Remote-US
Job number    5 added - Data Scientist
Job number    6 added - Data Scientist - 2022-1087
Job number    7 added - Lead Data Scientist (Remote)
Job number    8 added - Data Scientist, University Graduate (Data Analysis TikTok)- 2023 Start (BS/MS)
Job number    9 added - Data Scientist
Job number   10 added - Data Scientist
Job number   11 added - Data Scientist / Facilities
Job number   12 added - Data Scientist
Job number   13 added - Data Scientist
Job number   14 added - Data Scientist
Job number   15 added - Entry Level Data Scientist - Machine Learning Algorithms - 333322
Job number   16 added - Data Scientist
Job number   17 added - Data Scientist / Facilities
Job number   18 added - Entry Level Data Scientist - Machine Learning Algorithms - 333322
Job number   19 added - Data Scientist
Job number   20 added

In [16]:
driver.quit()

In [17]:
Links_list = dataframe['Links'].tolist()
#Links_list

In [18]:
import random
import time

In [19]:
driver = webdriver.Chrome(executable_path=driver_path)
descriptions=[]
for i in Links_list:
    driver.get(i)
    driver.implicitly_wait(random.randint(3, 8))
    jd = driver.find_element(By.XPATH, '//div[@id="jobDescriptionText"]').text
    descriptions.append(jd)
    time.sleep(random.randint(5,10))

dataframe['Descriptions'] = descriptions

C:\Users\ech0l\AppData\Local\Temp\ipykernel_1276\1391923102.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)


In [20]:
driver.quit()

In [21]:
# Convert the dataframe to a csv file
date = datetime.today().strftime('%Y-%m-%d')
dataframe.to_csv(date + "_" + position + "_" + locations + ".csv", index=False)

In [22]:
dataframe

,Title,Company,Location,Rating,Date,Salary,Description,Links,Descriptions
0,Jr. Data Scientist,Talentheed Inc,Remote,4.6,PostedToday,"$56,951 - $119,187 a year","To apply to data sets, create unique data mode...",https://www.indeed.com/company/Talentheed-Inc/...,Responsibilities : -\nCoordinate with differen...
1,Jr. Data Scientist,AffixITPro,Remote,NaN,PostedToday,$30 - $40 an hour,The Data Scientist supports the development of...,https://www.indeed.com/company/AffixITPro/jobs...,Job Description\nAffixIT Pro has been awarded ...
2,Data Scientist,BP,"Denver, CO 80201",3.8,PostedPosted 5 days ago,NaN,"In conjunction with Data Engineers, building a...",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Job Profile Summary\nResponsible for providing...
3,Data Scientist | Remote-US,IXIS Digital,"Remote in Burlington, VT 05401",NaN,PostedPosted 2 days ago,NaN,"Experience working with real-world datasets, i...",https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,"Overview\nIXIS is seeking an energetic, innova..."
4,Data Scientist,Macquarie Group Limited,"Philadelphia, NY",3.8,PostedPosted 30+ days ago,"$108,000 - $192,500 a year",Deep practical expertise applying statistical ...,https://www.indeed.com/rc/clk?jk=2222ecfa7ea02...,Join our innovative and collaborative Digital ...
...,...,...,...,...,...,...,...,...,...
2982,"Senior Data Analyst, Consumer Lending Credit P...",PenFed Credit Union,"McLean, VA 22102+2 locations",3.1,PostedPosted 30+ days ago,NaN,Identify required data and work with data stew...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Overview\n\nAre you looking to take your caree...
2983,"Senior Analyst, Data Reporting and Analytics",NATIONAL GRID CO USA (NE POWER),"Waltham, MA 02454+1 location",3.7,PostedPosted 10 days ago,NaN,Advocate for data analytics value across a bus...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,About us\nNational Grid is hiring a Senior Ana...
2984,Head of Machine Learning,Gables Search Group,"Pittsburgh, PA",5.0,PostedPosted 30+ days ago,NaN,Analyze various large-scale datasets including...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,My client is expanding and seeks a Head of Mac...
2985,Senior Statistical Programmer - United States,AL2S3 LTD,United States,NaN,PostedPosted 8 days ago,"$135,000 a year",4-6+ years of experience successfully performi...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Job Description\n\nGeorge Higginson is searchi...


In [37]:
date = datetime.today().strftime('%Y-%m-%d')
dataframe.to_csv(date + "_" + position + "_" + locations + ".csv", index=False)